## 타이타닉 데이터셋으로 전처리 연습

In [1]:
import numpy as np
import pandas as pd
# 만약 pasdas설치 안되어있으면
# !pip install pandas 치면됨!!

In [2]:
train = pd.read_csv('https://bit.ly/fc-ml-titanic') # 구글드라이브에 올려놓은 자료

In [3]:
# csv 파일에서 불러온 데이터셋의 가장 앞의 5개 보여줌
train.head() #안에 숫자n을 넣으면, 앞의 n개의 데이터 보여줌

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


* PassengerId: 승객 탑승 아이디
* Survived: 생존여부, 1: 생존, 0: 사망
* Pclass: 선실의 등급
* Name: 이름
* Sex: 성별
* Age: 나이
* SibSp: 형제, 자매, 배우자 수 (수평관계)
* Parch: 부모, 자식 수 (위아래 수직관계)
* TIcket: 티켓 번호
* Fare: 요금
* Cabin: 좌석 번호
* Embarked: 탑승 항구

### 전처리: train/validation 데이터셋 나누기

1. feature와 label을 정의
2. feature/label을 적절한 비율로 나눈다. 

In [4]:
# 전체 데이터 중 학습에 사용할 데이터를 선택, x로 사용
feature = ['Pclass', 'Sex', 'Age', 'Fare'] # x로 사용

# 정답값 혹은 예측값과 비교할 값, y로 사용
label = ['Survived'] 

In [5]:
# feature를 정의하고 train[feature]를 지정해주면 리스트안에 선택된 항목만 가져온다.
train[feature].head()

,Pclass,Sex,Age,Fare
0,3,male,22.0,7.2500
1,1,female,38.0,71.2833
2,3,female,26.0,7.9250
3,1,female,35.0,53.1000
4,3,male,35.0,8.0500


In [6]:
train[label].head() # 1: 생존, 0: 사망

,Survived
0,0
1,1
2,1
3,1
4,0


In [7]:
# train의 데이터 특성을 확인해보자
train.info() # 실행결과: Non-Null Count에 Age, Cabin은 데이터 누락됨을 알 수 있음

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [8]:
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(train[feature], train[label], test_size=0.2, random_state=123)

In [9]:
print(x_train.shape)
print(y_train.shape)
print(x_valid.shape)
print(y_valid.shape)

(712, 4)
(712, 1)
(179, 4)
(179, 1)


# 전처리(Preprocess): 결측치를 처리

In [10]:
train.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [11]:
# pandas의 isnull() 메소드를 사용하여 결측치의 전체 갯수를 확인
# 결측치의 전체 갯수를 확인
train.isnull().sum() # isnull()은 비어있는지를 확인

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [12]:
train['Age'].isnull().sum() # 'Age'의 데이터에 대해서만

177

## 수치형 결측치 데이터에 대한 처리

수치형 데이터의 결측치는 대체로 평균값을 구한 뒤, 평균값으로 채워준다. 

In [13]:
# 0으로 채웠을 때
# describe()는 통계치 데이터를 보여준다.
train['Age'].fillna(0).describe() # fillna(0)은 'Age'의 비어있는 값을 0으로 채운다.

count    891.000000
mean      23.799293
std       17.596074
min        0.000000
25%        6.000000
50%       24.000000
75%       35.000000
max       80.000000
Name: Age, dtype: float64

In [14]:
# 평균값으로 채웠을 때, 
meanAge = train['Age'].mean()
print(meanAge)
train['Age'].fillna(meanAge).describe()

# 실행결과보면, mean값이 0으로 채웠을 때보다 평균값으로 채웠을 떄 더 올라가.(당연한 결과)

29.69911764705882


count    891.000000
mean      29.699118
std       13.002015
min        0.420000
25%       22.000000
50%       29.699118
75%       35.000000
max       80.000000
Name: Age, dtype: float64

### scikit-learn에서 imputer 메서드를 활용하면, 2개 이상의 컬럼을 한번에 처리할 수 있다.

In [15]:
from sklearn.impute import SimpleImputer
# 순서 외우기: imputer 모델 선언 → fit() → transform()
# imputer 모델 선언
imputer = SimpleImputer(strategy = 'mean')

In [16]:
# fit()
imputer.fit(train[['Age', 'Pclass']])

SimpleImputer()

In [17]:
# transform()
result = imputer.transform(train[['Age', 'Pclass']])

In [18]:
result

array([[22.        ,  3.        ],
       [38.        ,  1.        ],
       [26.        ,  3.        ],
       ...,
       [29.69911765,  3.        ],
       [26.        ,  1.        ],
       [32.        ,  3.        ]])

In [19]:
train[['Age', 'Pclass']] = result

In [20]:
# Age의 null이 없어진걸 확인할 수 있음
train[['Age', 'Pclass']].isnull().sum()

Age       0
Pclass    0
dtype: int64

In [21]:
train[['Age', 'Pclass']].describe()

,Age,Pclass
count,891.000000,891.000000
mean,29.699118,2.308642
std,13.002015,0.836071
min,0.420000,1.000000
25%,22.000000,2.000000
50%,29.699118,3.000000
75%,35.000000,3.000000
max,80.000000,3.000000


In [22]:
# 결측치가 있던 원본데이터로 다시 복귀
train = pd.read_csv('https://bit.ly/fc-ml-titanic')

In [23]:
train[['Age', 'Pclass']].isnull().sum() # 실행결과로 Age에 null이 177개 있음을 알수있음

Age       177
Pclass      0
dtype: int64

In [24]:
# imputer model정의
imputer = SimpleImputer(strategy = 'mean')

In [25]:
# 아까는 fit() + transform() 을 각각 실행했다면,
# 이번엔 fit_transform()함수로 한꺼번에 실행
result = imputer.fit_transform(train[['Age', 'Pclass']])

In [26]:
train[['Age', 'Pclass']] = result

In [27]:
train[['Age', 'Pclass']].isnull().sum()

Age       0
Pclass    0
dtype: int64

In [28]:
train[['Age', 'Pclass']].describe()

,Age,Pclass
count,891.000000,891.000000
mean,29.699118,2.308642
std,13.002015,0.836071
min,0.420000,1.000000
25%,22.000000,2.000000
50%,29.699118,3.000000
75%,35.000000,3.000000
max,80.000000,3.000000


## 참고로, mean 대신 median, most_frequent, constant를 넣을 수도 있음
## 예) imputer = SimpleImputer(strategy = 'median')

# Categorical 데이터에 대한 결측치 처리

In [29]:
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [30]:
# 위에서 Embarked를 처리
train['Embarked']

0      S
1      C
2      S
3      S
4      S
      ..
886    S
887    S
888    S
889    C
890    Q
Name: Embarked, Length: 891, dtype: object

In [31]:
train['Embarked'].fillna('S') # null값에 'S'로 채움

0      S
1      C
2      S
3      S
4      S
      ..
886    S
887    S
888    S
889    C
890    Q
Name: Embarked, Length: 891, dtype: object

In [32]:
# imputer model 선언
imputer = SimpleImputer(strategy = 'most_frequent') # 여기서는 'S'가 most_frequent임

In [33]:
result = imputer.fit_transform(train[['Embarked', 'Cabin']])

In [34]:
train[['Embarked', 'Cabin']] = result

In [35]:
train[['Embarked', 'Cabin']].isnull().sum()

Embarked    0
Cabin       0
dtype: int64

# Label Encoding: 문자를 수치로 변환

학습을 할 때는 문자로된 데이터를 수치로 변환해야함

In [36]:
def convert(data):
    if data == 'male':
        return 1
    elif data == 'female':
        return 0

In [37]:
train['Sex'].value_counts()

male      577
female    314
Name: Sex, dtype: int64

In [38]:
train['Sex'].apply(convert) # apply를 이용해, train['Sex'] 값들을 convert라는 함수에 입력값으로 넣음

0      1
1      0
2      0
3      0
4      1
      ..
886    1
887    0
888    0
889    1
890    1
Name: Sex, Length: 891, dtype: int64

In [39]:
from sklearn.preprocessing import LabelEncoder

# LabelEncoder 모델 정의
le = LabelEncoder()

In [40]:
# fit_transform()
train['Sex_num'] = le.fit_transform(train['Sex']) # train안에 'Sex_num'열이 늘어남

In [41]:
le.classes_

array(['female', 'male'], dtype=object)

In [42]:
le.inverse_transform([0,1,1,0])

array(['female', 'male', 'male', 'female'], dtype=object)

In [43]:
# 'S', 'C', 'Q' class
train['Embarked_num'] = le.fit_transform(train['Embarked'])

In [44]:
train['Embarked_num']

0      2
1      0
2      2
3      2
4      2
      ..
886    2
887    2
888    2
889    0
890    1
Name: Embarked_num, Length: 891, dtype: int32

In [45]:
le.classes_ # ['C', 'Q', 'S'] 순서로 0, 1, 2

array(['C', 'Q', 'S'], dtype=object)

원-핫 엔코딩(One-hot Encoding)

In [46]:
# 결측치가 있던 원본데이터로 다시 복귀
train = pd.read_csv('https://bit.ly/fc-ml-titanic')

In [47]:
train['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [48]:
train['Embarked'][:6] # 앞의 6행만 보여줌

0    S
1    C
2    S
3    S
4    S
5    Q
Name: Embarked, dtype: object

# 순서: 문자열 -> labelEncoder() 변환 -> one-hot Encoding 
# 여기서 에러뜸: TypeError: Encoders require their input to be uniformly strings or numbers. Got ['float', 'str']
train['Embarked_num'] = LabelEncoder().fit_transform(train['Embarked'])

# 전처리: Normalize(정규화)
ex. 영화평점 리뷰
- 네이버에서 평점 리뷰 기준이 0점~10점-> 데이터가 [2, 4, 6, 8, 10]
- 넷플릭스 평점 리뷰 기준이 0점~5점-> 데이터가 [1, 2, 3, 4, 5]

데이터간에 다른 min, max값을 가지는 경우, 정규화를 통해 최소/최대치 값의 척도를 맞추어준다.

In [52]:
movie = {'naver': [2, 4, 6, 8, 10], 'netflix': [1, 2, 3, 4, 5]}

In [53]:
import pandas as pd
movies = pd.DataFrame(data=movie)
movies

,naver,netflix
0,2,1
1,4,2
2,6,3
3,8,4
4,10,5


In [54]:
from sklearn.preprocessing import MinMaxScaler

In [55]:
# 모델선언
min_max_scaler = MinMaxScaler()

In [56]:
min_max_movie = min_max_scaler.fit_transform(movies)

In [58]:
# min =0, max = 1 정규분포가 0~1사이로 바뀐다. 
pd.DataFrame(min_max_movie, columns=['naver', 'netflix'])

,naver,netflix
0,0.00,0.00
1,0.25,0.25
2,0.50,0.50
3,0.75,0.75
4,1.00,1.00
